In [1]:
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import notebook
import json
from collections import Counter
from plotnine import *
import plotnine

In [2]:
import matplotlib.font_manager as fm
import matplotlib as mpl
import koreanize_matplotlib
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters

font_path = 'C:/Users/User/Desktop/URP/NanumBarunGothic.ttf'
fe = fm.FontEntry(fname=font_path,name='font')
fm.fontManager.ttflist.insert(0,fe)
plt.rcParams.update({'font.size':12,'font.family':'NanumGothic'})

In [3]:
%%time
# 입력데이터 로드
train = pd.read_json('train.json', typ = 'frame')
test = pd.read_json('test.json', typ = 'frame')
val = pd.read_json('val.json', typ = 'frame')
genre = pd.read_json('genre_gn_all.json', typ = 'series')
meta = pd.read_json('song_meta.json', typ = 'frame')

CPU times: total: 10.5 s
Wall time: 10.8 s


In [5]:
# paper baseline 구현 위해 new split 시도
!python try_new_split.py run train.json

Reading data...

Total playlists: 115071
Discard playlists with less than 5 tracks...



Traceback (most recent call last):
  File "c:\Users\User\Desktop\URP\code\try_new_split.py", line 163, in <module>
    fire.Fire(ArenaSplitter)
  File "c:\Users\User\anaconda3\envs\env1\Lib\site-packages\fire\core.py", line 141, in Fire
    component_trace = _Fire(component, args, parsed_flag_args, context, name)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\User\anaconda3\envs\env1\Lib\site-packages\fire\core.py", line 475, in _Fire
    component, remaining_args = _CallAndUpdateTrace(
                                ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\User\anaconda3\envs\env1\Lib\site-packages\fire\core.py", line 691, in _CallAndUpdateTrace
    component = fn(*varargs, **kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\User\Desktop\URP\code\try_new_split.py", line 140, in run
    playlst_song_map = self._mapping_function(playlists, 'id', 'songs')
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  

In [3]:
!python split_data.py run train.json

Reading data...

Total playlists: 115071
Splitting data...
Original train...
Original val...
Masked val...
Total: 23015, Song only: 6904, Song & Tags: 11508, Tags only: 3452, Title only: 1151


In [5]:
# 주어진 전체 플레이리스트에서 가장 많이 등장한 곡과 태그를 모든 문제에 대해서 답안으로 내놓는 모델

!python most_popular.py run \
 	--train_fname=arena_data/orig/train.json \
 	--question_fname=arena_data/questions/val.json


Loading train file...
Loading question file...
Writing answers...



100%|██████████| 23015/23015 [00:00<00:00, 37251.53it/s]


In [11]:
# 주어지는 각 문제마다, 가장 많이 등장하는 장르에 대해 해당 장르에서 가장 빈번하게 등장하는 곡들을 답안으로 내놓는 모델

!python genre_most_popular.py run \
 	--song_meta_fname=song_meta.json \
 	--train_fname=arena_data/orig/train.json \
 	--question_fname=arena_data/questions/val.json

Loading song meta...
Loading train file...
Loading question file...
Writing answers...



100%|██████████| 23015/23015 [00:01<00:00, 13346.56it/s]


In [7]:
# SVD

!python MF.py run \
 	--song_meta_fname=song_meta.json \
 	--train_fname=arena_data/orig/train.json \
 	--question_fname=arena_data/questions/val.json

Loading song meta...
Loading train file...
Loading question file...
Writing answers...


Traceback (most recent call last):
  File "c:\Users\User\Desktop\URP\code\MF.py", line 68, in <module>
    fire.Fire(MFmax)
  File "c:\Users\User\anaconda3\envs\env1\Lib\site-packages\fire\core.py", line 141, in Fire
    component_trace = _Fire(component, args, parsed_flag_args, context, name)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\User\anaconda3\envs\env1\Lib\site-packages\fire\core.py", line 475, in _Fire
    component, remaining_args = _CallAndUpdateTrace(
                                ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\User\anaconda3\envs\env1\Lib\site-packages\fire\core.py", line 691, in _CallAndUpdateTrace
    component = fn(*varargs, **kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\User\Desktop\URP\code\MF.py", line 63, in run
    answers = self._generate_answers(song_meta_json, train_data, questions)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\User\Desk

In [8]:
# Random selection

!python random_select.py run \
 	--train_fname=arena_data/orig/train.json \
 	--question_fname=arena_data/questions/val.json

Loading train file...
Loading question file...
Writing answers...



100%|██████████| 23015/23015 [00:00<00:00, 35916.52it/s]


In [8]:
# Score 평가

!python evaluate.py evaluate \
	--gt_fname=arena_data/answers/val.json \
	--rec_fname=arena_data/results/results.json

Music nDCG: 0.0169694
Music MAP: 0.0239726
